# Loading up the Accounting Deferred Workbook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import pickle

### Loading the historical deferred balances by BU and Major Product Config

##### NOTE: The historical data includes Marketo and Magento (whereas these are not included on the other sheets

Need to make the following changes
 - 'External Reporting BU' needs to be billed forward
 - Totals need to be eliminated
 - delete unnecessary rows
 
 ###### Note: There are excess rows in the dataframe for digital media subscriptions that are hidden. Need to remove
  - There are leading zeros in the BUs and MPCs
  

# FOR TODAY 6/8/20 Skip the whole deferred trend. Just get to the forecast

In [2]:
#df = pd.read_excel("../data/Data_2020_P06/Q2'20 Rev Acctg Mgmt Workbook (06-04-20).xlsx", 
#                   sheet_name='Deferred Revenue Trend', skiprows=5)

In [3]:
def clean_DR_history(df):
    
    df = df.rename(columns={'External Reporting BU': 'BU',
                       'Major Product Config': 'MPC',
                       'Q/Q change': 'q_over_q',
                       'Y/Y change': 'y_over_y'})
    df = df.drop(columns = ['Unnamed: 13', 'Unnamed: 14'])
    
    df['BU'] = df['BU'].fillna(method='ffill')
    
    df['BU'] = df['BU'].str.strip()
    df['MPC'] = df['MPC'].str.strip()

    df = df.dropna(subset=['MPC'], axis=0)
    df = df.drop_duplicates(subset=['BU','MPC'], keep='first')
    df = df.set_index(['BU', 'MPC'])
    
    old_columns = df.columns
    new_columns = []
    for col in old_columns:
        new_columns.append(col.replace("'", '_'))
        
    columns_dict = dict(zip(old_columns, new_columns))
    df.rename(columns = columns_dict, inplace=True)
    
    return df

In [ ]:
#df_hist = clean_DR_history(df)

In [ ]:
#df_hist.head(40)

### Loading the Deferred Revenue Forecast Sheet

#### Note: For today, simply take the totals by each BU by quarter and move on!

To be done
 - clear out rows below the Grand Total inclusive of Magento/Marketo
 - forward fill the External Reporting BU
 - Move Marketo and Magento BU to Digital Experience
 - Aggregate this by External reporting BU
 - rename columns without that '
 - create interpolated periods here for the amortization (assume amortization to revenue is linear within the periods of a quarter
 - Save and move on
 
##### Note: There are a lot of rows at the bottom that need to be cleaned up

In [4]:
df = pd.read_excel("../data/Data_2020_P06/Q2'20 Rev Acctg Mgmt Workbook (06-04-20).xlsx", 
                   sheet_name='Deferred Revenue Forecast', skiprows=5)

In [5]:
df.head(50)

,External Reporting BU,Major Product Config,Historical,Q3'20,Q4'20,Q1'21,Q2'21,Q3'21,Q4'21,Q1'22,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,Digital Media,Consulting,0,1022.47,748.575,29.777,3.149,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Educational,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Full,0,-0.278,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Licensing,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Licensing M&S,0,283.978,266.207,47.156,21.977,5.002,2.623,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Other,0,-26634.5,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Royalty,0,418.614,233.746,454.925,57.057,35.761,34.685,3.114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Royalty M&S,0,1028.3,827.391,690.155,475.079,29.679,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Software as Service,0,44835,31698.9,20254.3,6908.51,185.827,41.15,31.458,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Subscription,0,1.0296e+06,622760,361533,112857,11448,8117.2,5325.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.columns

Index(['External Reporting BU', 'Major Product Config', ' Historical', 'Q3'20',
       'Q4'20', 'Q1'21', 'Q2'21', 'Q3'21', 'Q4'21', 'Q1'22', 'Q2'22', 'Q3'22+',
       'As Performed / Upon Acceptance', 'Total', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50'],
      dtype='object')

##### Stripping spaces in the External Reporting BU columns

In [8]:
df['External Reporting BU'] = df['External Reporting BU'].str.strip()

##### Clearing out the rows below that we do not need

In [9]:
end_loc = df[df['External Reporting BU']=='Grand Total inclusive of Magento/Marketo']
end_index = end_loc.index[0]

df = df[df.index <= end_index]

In [10]:
df['External Reporting BU'].value_counts()

Digital Media                               1
Grand Total inclusive of Magento/Marketo    1
Marketo Deferred                            1
Publishing Total                            1
Magento Deferred                            1
Publishing                                  1
Digital Media Total                         1
Digital Experience Total                    1
Grand Total                                 1
Digital Experience                          1
Name: External Reporting BU, dtype: int64

### We are just taking the following rows
- Digital Media Total
- Publishing Total
- Digital Experience Total
- Marketo Deferred
- Magento Deferred

Then we need to add the Marketo and Magento defered to the digital experience total


In [11]:
keeper_rows = ['Digital Media Total',
              'Publishing Total',
              'Digital Experience Total',
              'Marketo Deferred',
              'Magento Deferred',
              'Grand Total inclusive of Magento/Marketo']

df_test = df[df['External Reporting BU'].isin(keeper_rows)]
df_test.head(10)

,External Reporting BU,Major Product Config,Historical,Q3'20,Q4'20,Q1'21,Q2'21,Q3'21,Q4'21,Q1'22,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
13,Digital Media Total,NaN,0,1.0519e+06,657534,383776,120701,11924.1,8320,5423.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Digital Experience Total,NaN,-120,389900,240669,138374,38697.4,15424.2,10513.4,3010.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Publishing Total,NaN,0,30997.3,19928.3,11714.3,5337.42,3280.49,807.988,347.025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,Magento Deferred,NaN,NaN,32808.4,21486.5,12543.9,5166.5,1310.1,926.829,476.411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,Marketo Deferred,NaN,NaN,76774.6,43148.6,22904.5,7524.18,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Grand Total inclusive of Magento/Marketo,NaN,-120,1.58238e+06,982766,569312,177427,31938.9,20568.2,9256.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning out bad columns

In [12]:
df_test.columns

Index(['External Reporting BU', 'Major Product Config', ' Historical', 'Q3'20',
       'Q4'20', 'Q1'21', 'Q2'21', 'Q3'21', 'Q4'21', 'Q1'22', 'Q2'22', 'Q3'22+',
       'As Performed / Upon Acceptance', 'Total', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50'],
      dtype='object')

In [13]:
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]
df_test = df_test.drop(columns= ['Major Product Config', ' Historical'])

In [14]:
df_test.head(10)

,External Reporting BU,Q3'20,Q4'20,Q1'21,Q2'21,Q3'21,Q4'21,Q1'22,Q2'22,Q3'22+,As Performed / Upon Acceptance,Total
13,Digital Media Total,1.0519e+06,657534,383776,120701,11924.1,8320,5423.13,3864.81,64924.4,5601.75,2.31397e+06
23,Digital Experience Total,389900,240669,138374,38697.4,15424.2,10513.4,3010.43,1871.82,4073.25,730.422,843264
36,Publishing Total,30997.3,19928.3,11714.3,5337.42,3280.49,807.988,347.025,133.832,137.54,0,72684.2
38,Magento Deferred,32808.4,21486.5,12543.9,5166.5,1310.1,926.829,476.411,596.116,943.308,183.437,76441.5
39,Marketo Deferred,76774.6,43148.6,22904.5,7524.18,0,0,0,0,0,4158.31,154510
40,Grand Total inclusive of Magento/Marketo,1.58238e+06,982766,569312,177427,31938.9,20568.2,9256.99,6466.58,70078.5,10673.9,3.46087e+06


## Add Magento and Marketo to Digital Experience

##### NOTE: The External Reporting BU is different the the BU we have in deferred.
We will have to combine Creative and Document Cloud to get to the External Reporting BU since both show up as Digital Media in this accounting workbook


In [15]:
df_test['External Reporting BU'] = df_test['External Reporting BU'].str.replace('Magento Deferred', 
                                                                               'Digital Experience Total')

In [16]:
df_test['External Reporting BU'] = df_test['External Reporting BU'].str.replace('Marketo Deferred', 
                                                                               'Digital Experience Total')

In [1]:
# Was this repeated as a mistake?
#df_test['External Reporting BU'] = df_test['External Reporting BU'].str.replace('Marketo Deferred', 
#                                                                               'Digital Experience Total')

In [18]:
changed_columns = df_test.columns.str.replace("'", '_')
changed_columns = changed_columns.str.replace('+', "")

In [19]:
df_test.columns = changed_columns

In [20]:
# start by renaming the columns

In [21]:
df_test.columns

Index(['External Reporting BU', 'Q3_20', 'Q4_20', 'Q1_21', 'Q2_21', 'Q3_21',
       'Q4_21', 'Q1_22', 'Q2_22', 'Q3_22', 'As Performed / Upon Acceptance',
       'Total'],
      dtype='object')

In [22]:
df_test

,External Reporting BU,Q3_20,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,As Performed / Upon Acceptance,Total
13,Digital Media,1.0519e+06,657534,383776,120701,11924.1,8320,5423.13,3864.81,64924.4,5601.75,2.31397e+06
23,Digital Experience,389900,240669,138374,38697.4,15424.2,10513.4,3010.43,1871.82,4073.25,730.422,843264
36,Publishing,30997.3,19928.3,11714.3,5337.42,3280.49,807.988,347.025,133.832,137.54,0,72684.2
38,Digital Experience,32808.4,21486.5,12543.9,5166.5,1310.1,926.829,476.411,596.116,943.308,183.437,76441.5
39,Digital Experience,76774.6,43148.6,22904.5,7524.18,0,0,0,0,0,4158.31,154510
40,Grand inclusive of Magento/Marketo,1.58238e+06,982766,569312,177427,31938.9,20568.2,9256.99,6466.58,70078.5,10673.9,3.46087e+06


In [23]:
df_test_gb = df_test.groupby('External Reporting BU').sum()

In [24]:
df_test_gb

,Q3_20,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,As Performed / Upon Acceptance,Total
External Reporting BU,,,,,,,,,,,
Digital Experience,4.994834e+05,305303.73931,173822.55227,51388.09531,16734.29407,11440.21483,3486.83955,2467.93488,5016.553322,5072.17166,1.074216e+06
Digital Media,1.051898e+06,657534.01100,383775.58300,120701.44900,11924.11300,8319.99900,5423.12700,3864.80900,64924.390000,5601.74500,2.313967e+06
Grand inclusive of Magento/Marketo,1.582378e+06,982766.07931,569312.46127,177426.96231,31938.89607,20568.20183,9256.99155,6466.57588,70078.483322,10673.91666,3.460867e+06
Publishing,3.099728e+04,19928.32900,11714.32600,5337.41800,3280.48900,807.98800,347.02500,133.83200,137.540000,0.00000,7.268423e+04


### Now that we have the data that is all numeric, we need to adjust for the reporting in thousands (FP&A report)

In [25]:
df_test_gb = df_test_gb * 1000

In [26]:
df_test_gb

,Q3_20,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,As Performed / Upon Acceptance,Total
External Reporting BU,,,,,,,,,,,
Digital Experience,4.994834e+08,3.053037e+08,1.738226e+08,5.138810e+07,16734294.07,11440214.83,3486839.55,2467934.88,5.016553e+06,5.072172e+06,1.074216e+09
Digital Media,1.051898e+09,6.575340e+08,3.837756e+08,1.207014e+08,11924113.00,8319999.00,5423127.00,3864809.00,6.492439e+07,5.601745e+06,2.313967e+09
Grand inclusive of Magento/Marketo,1.582378e+09,9.827661e+08,5.693125e+08,1.774270e+08,31938896.07,20568201.83,9256991.55,6466575.88,7.007848e+07,1.067392e+07,3.460867e+09
Publishing,3.099728e+07,1.992833e+07,1.171433e+07,5.337418e+06,3280489.00,807988.00,347025.00,133832.00,1.375400e+05,0.000000e+00,7.268423e+07


### Creating the columns that have this amortization by period

# Below here is the first mistake - need to create a P00 column and then delete it! Such that the amortization does not include the actual period that is in there

In [29]:
new_columns = []
for i in range(12*3):
    if len(str(i))==1:
        new_column = 'P0'+ str(i)
    else:
        new_column = 'P'+ str(i)
    new_columns.append(new_column)

In [30]:
new_columns

['P00',
 'P01',
 'P02',
 'P03',
 'P04',
 'P05',
 'P06',
 'P07',
 'P08',
 'P09',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P19',
 'P20',
 'P21',
 'P22',
 'P23',
 'P24',
 'P25',
 'P26',
 'P27',
 'P28',
 'P29',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35']

In [31]:
qtrly_list = [col for col in df_test_gb.columns if 'Q' in col]
qtrly_list

['Q3_20',
 'Q4_20',
 'Q1_21',
 'Q2_21',
 'Q3_21',
 'Q4_21',
 'Q1_22',
 'Q2_22',
 'Q3_22']

In [32]:
period_index = 0
for index, qtr in enumerate(qtrly_list):
    
    df_test_gb[new_columns[period_index]]=df_test_gb[qtr] / 3
    period_index +=1
    df_test_gb[new_columns[period_index]]=df_test_gb[qtr] / 3
    period_index +=1
    df_test_gb[new_columns[period_index]]=df_test_gb[qtr] / 3
    period_index +=1


In [33]:
df_test_gb

,Q3_20,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,As Performed / Upon Acceptance,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience,4.994834e+08,3.053037e+08,1.738226e+08,5.138810e+07,16734294.07,11440214.83,3486839.55,2467934.88,5.016553e+06,5.072172e+06,...,3.813405e+06,1162279.85,1162279.85,1162279.85,8.226450e+05,8.226450e+05,8.226450e+05,1.672184e+06,1.672184e+06,1.672184e+06
Digital Media,1.051898e+09,6.575340e+08,3.837756e+08,1.207014e+08,11924113.00,8319999.00,5423127.00,3864809.00,6.492439e+07,5.601745e+06,...,2.773333e+06,1807709.00,1807709.00,1807709.00,1.288270e+06,1.288270e+06,1.288270e+06,2.164146e+07,2.164146e+07,2.164146e+07
Grand inclusive of Magento/Marketo,1.582378e+09,9.827661e+08,5.693125e+08,1.774270e+08,31938896.07,20568201.83,9256991.55,6466575.88,7.007848e+07,1.067392e+07,...,6.856067e+06,3085663.85,3085663.85,3085663.85,2.155525e+06,2.155525e+06,2.155525e+06,2.335949e+07,2.335949e+07,2.335949e+07
Publishing,3.099728e+07,1.992833e+07,1.171433e+07,5.337418e+06,3280489.00,807988.00,347025.00,133832.00,1.375400e+05,0.000000e+00,...,2.693293e+05,115675.00,115675.00,115675.00,4.461067e+04,4.461067e+04,4.461067e+04,4.584667e+04,4.584667e+04,4.584667e+04


## I Don't need the everything in this. I can now remove some of the details

First check that my periods match the quarterly deferred numbers/


In [34]:
df_qtrly_only = df_test_gb.copy()
df_period_only = df_test_gb.copy()

df_period_only = df_period_only.loc[:, df_period_only.columns.str.contains('P')]
df_qtrly_only = df_qtrly_only.loc[:, ~df_qtrly_only.columns.str.contains('P')]
                                  
df_period_only['total'] = df_period_only.sum(axis=1)
                            

In [35]:
df_period_only

,As Performed / Upon Acceptance,P00,P01,P02,P03,P04,P05,P06,P07,P08,...,P18,P19,P20,P21,P22,P23,P24,P25,P26,total
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience,5.072172e+06,1.664945e+08,1.664945e+08,1.664945e+08,1.017679e+08,1.017679e+08,1.017679e+08,5.794085e+07,5.794085e+07,5.794085e+07,...,1162279.85,1162279.85,1162279.85,8.226450e+05,8.226450e+05,8.226450e+05,1.672184e+06,1.672184e+06,1.672184e+06,1.074216e+09
Digital Media,5.601745e+06,3.506325e+08,3.506325e+08,3.506325e+08,2.191780e+08,2.191780e+08,2.191780e+08,1.279252e+08,1.279252e+08,1.279252e+08,...,1807709.00,1807709.00,1807709.00,1.288270e+06,1.288270e+06,1.288270e+06,2.164146e+07,2.164146e+07,2.164146e+07,2.313967e+09
Grand inclusive of Magento/Marketo,1.067392e+07,5.274594e+08,5.274594e+08,5.274594e+08,3.275887e+08,3.275887e+08,3.275887e+08,1.897708e+08,1.897708e+08,1.897708e+08,...,3085663.85,3085663.85,3085663.85,2.155525e+06,2.155525e+06,2.155525e+06,2.335949e+07,2.335949e+07,2.335949e+07,3.460867e+09
Publishing,0.000000e+00,1.033243e+07,1.033243e+07,1.033243e+07,6.642776e+06,6.642776e+06,6.642776e+06,3.904775e+06,3.904775e+06,3.904775e+06,...,115675.00,115675.00,115675.00,4.461067e+04,4.461067e+04,4.461067e+04,4.584667e+04,4.584667e+04,4.584667e+04,7.268423e+07


In [36]:
df_qtrly_only

,Q3_20,Q4_20,Q1_21,Q2_21,Q3_21,Q4_21,Q1_22,Q2_22,Q3_22,Total
External Reporting BU,,,,,,,,,,
Digital Experience,4.994834e+08,3.053037e+08,1.738226e+08,5.138810e+07,16734294.07,11440214.83,3486839.55,2467934.88,5.016553e+06,1.074216e+09
Digital Media,1.051898e+09,6.575340e+08,3.837756e+08,1.207014e+08,11924113.00,8319999.00,5423127.00,3864809.00,6.492439e+07,2.313967e+09
Grand inclusive of Magento/Marketo,1.582378e+09,9.827661e+08,5.693125e+08,1.774270e+08,31938896.07,20568201.83,9256991.55,6466575.88,7.007848e+07,3.460867e+09
Publishing,3.099728e+07,1.992833e+07,1.171433e+07,5.337418e+06,3280489.00,807988.00,347025.00,133832.00,1.375400e+05,7.268423e+07


##### OK My periods work fine. Now I can move on to saving this and finishing the defered waterfall

In [37]:
df_test_gb.columns

Index(['Q3_20', 'Q4_20', 'Q1_21', 'Q2_21', 'Q3_21', 'Q4_21', 'Q1_22', 'Q2_22',
       'Q3_22', 'As Performed / Upon Acceptance', 'Total', 'P00', 'P01', 'P02',
       'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12',
       'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22',
       'P23', 'P24', 'P25', 'P26'],
      dtype='object')

In [38]:
df_waterfall = df_test_gb.loc[:, df_test_gb.columns.str.contains('P')]

In [39]:
df_waterfall

,As Performed / Upon Acceptance,P00,P01,P02,P03,P04,P05,P06,P07,P08,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience,5.072172e+06,1.664945e+08,1.664945e+08,1.664945e+08,1.017679e+08,1.017679e+08,1.017679e+08,5.794085e+07,5.794085e+07,5.794085e+07,...,3.813405e+06,1162279.85,1162279.85,1162279.85,8.226450e+05,8.226450e+05,8.226450e+05,1.672184e+06,1.672184e+06,1.672184e+06
Digital Media,5.601745e+06,3.506325e+08,3.506325e+08,3.506325e+08,2.191780e+08,2.191780e+08,2.191780e+08,1.279252e+08,1.279252e+08,1.279252e+08,...,2.773333e+06,1807709.00,1807709.00,1807709.00,1.288270e+06,1.288270e+06,1.288270e+06,2.164146e+07,2.164146e+07,2.164146e+07
Grand inclusive of Magento/Marketo,1.067392e+07,5.274594e+08,5.274594e+08,5.274594e+08,3.275887e+08,3.275887e+08,3.275887e+08,1.897708e+08,1.897708e+08,1.897708e+08,...,6.856067e+06,3085663.85,3085663.85,3085663.85,2.155525e+06,2.155525e+06,2.155525e+06,2.335949e+07,2.335949e+07,2.335949e+07
Publishing,0.000000e+00,1.033243e+07,1.033243e+07,1.033243e+07,6.642776e+06,6.642776e+06,6.642776e+06,3.904775e+06,3.904775e+06,3.904775e+06,...,2.693293e+05,115675.00,115675.00,115675.00,4.461067e+04,4.461067e+04,4.461067e+04,4.584667e+04,4.584667e+04,4.584667e+04


### Now Drop P00 from the waterfall

In [40]:
df_waterfall = df_waterfall.drop('P00', axis=1)

In [41]:
df_waterfall

,As Performed / Upon Acceptance,P01,P02,P03,P04,P05,P06,P07,P08,P09,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience,5.072172e+06,1.664945e+08,1.664945e+08,1.017679e+08,1.017679e+08,1.017679e+08,5.794085e+07,5.794085e+07,5.794085e+07,1.712937e+07,...,3.813405e+06,1162279.85,1162279.85,1162279.85,8.226450e+05,8.226450e+05,8.226450e+05,1.672184e+06,1.672184e+06,1.672184e+06
Digital Media,5.601745e+06,3.506325e+08,3.506325e+08,2.191780e+08,2.191780e+08,2.191780e+08,1.279252e+08,1.279252e+08,1.279252e+08,4.023382e+07,...,2.773333e+06,1807709.00,1807709.00,1807709.00,1.288270e+06,1.288270e+06,1.288270e+06,2.164146e+07,2.164146e+07,2.164146e+07
Grand inclusive of Magento/Marketo,1.067392e+07,5.274594e+08,5.274594e+08,3.275887e+08,3.275887e+08,3.275887e+08,1.897708e+08,1.897708e+08,1.897708e+08,5.914232e+07,...,6.856067e+06,3085663.85,3085663.85,3085663.85,2.155525e+06,2.155525e+06,2.155525e+06,2.335949e+07,2.335949e+07,2.335949e+07
Publishing,0.000000e+00,1.033243e+07,1.033243e+07,6.642776e+06,6.642776e+06,6.642776e+06,3.904775e+06,3.904775e+06,3.904775e+06,1.779139e+06,...,2.693293e+05,115675.00,115675.00,115675.00,4.461067e+04,4.461067e+04,4.461067e+04,4.584667e+04,4.584667e+04,4.584667e+04


### Saving the waterfall as Q2_waterfall

In [42]:
pickle.dump(df_waterfall, open('../data/processed/Q2_waterfall.p', 'wb'))

### This will get loaded into the deferred revenue forecast